In [22]:
# add utils to path
import sys
sys.path.append('../')

import json
import numpy as np

In [23]:
def quaternion_multiply(q1, q2):
    # Unpack the quaternions
    w1, x1, y1, z1 = q1
    w2, x2, y2, z2 = q2
    
    # Perform the multiplication
    w = w1*w2 - x1*x2 - y1*y2 - z1*z2
    x = w1*x2 + x1*w2 + y1*z2 - z1*y2
    y = w1*y2 - x1*z2 + y1*w2 + z1*x2
    z = w1*z2 + x1*y2 - y1*x2 + z1*w2
    
    return [w, x, y, z]

# def rotate_around_z(R, T, theta, shift=[0.5, 0.5]):
#     # Convert theta from degrees to radians
#     theta_rad = np.radians(theta)
    
#     # Create a quaternion for rotation around the Z axis
#     q_rot_z = [np.cos(theta_rad / 2), 0, 0, np.sin(theta_rad / 2)]
#     q_rot_x = [np.cos(theta_rad / 2), np.sin(theta_rad / 2), 0, 0]
#     q_rot_y = [np.cos(theta_rad / 2), 0, np.sin(theta_rad / 2), 0]
#     q_rot = q_rot_y

#     T_shifted = [T[0] - shift[0], T[1] - shift[1], T[2]]
#     T_shifted_quat = [0] + T_shifted
#     T_rotated_shifted_quat = quaternion_multiply(
#         quaternion_multiply(q_rot, T_shifted_quat),
#         [q_rot[0], -q_rot[1], -q_rot[2], -q_rot[3]]
#     )

#     T_rotated = [
#         T_rotated_shifted_quat[1] + shift[0],
#         T_rotated_shifted_quat[2] + shift[1],
#         T_rotated_shifted_quat[3]
#     ]

#     # Rotate R
#     R_rotated = quaternion_multiply(q_rot, R)
    
    
#     return R_rotated, T_rotated

def rotate_around_z(R, T, theta, shift=[0.5, 0.5]):
    # Convert theta from degrees to radians
    theta_rad = np.radians(theta)
    
    # Create a quaternion for rotation around the Z axis
    q_rot_y = [np.cos(theta_rad / 2), 0, np.sin(theta_rad / 2), 0]
    q_rot = q_rot_y

    T_shifted = [T[0] - shift[0], T[1], T[2] - shift[1]]
    T_shifted_quat = [0] + T_shifted
    T_rotated_shifted_quat = quaternion_multiply(
        quaternion_multiply(q_rot, T_shifted_quat),
        [q_rot[0], -q_rot[1], -q_rot[2], -q_rot[3]]
    )

    T_rotated = [
        T_rotated_shifted_quat[1] + shift[0],
        T_rotated_shifted_quat[2],
        T_rotated_shifted_quat[3] + shift[1]
    ]

    # Rotate R
    R_rotated = quaternion_multiply(q_rot, R)
    
    
    return R_rotated, T_rotated

In [24]:
cam = json.load(open('../cam.json'))

In [25]:
cam

{'loop': False,
 'path': [{'R': [0.9380469918251038,
    -0.0429956391453743,
    0.3198738396167755,
    0.12608736753463745],
   'T': [-1.1177868843078613, 0.8019675016403198, 1.45546555519104],
   'aperture_size': 0.0,
   'fov': 50.625,
   'glow_mode': 0,
   'glow_y_cutoff': 0.0,
   'scale': 1.1269716024398804,
   'slice': 0.14271652698516846}],
 'time': 1.0}

In [26]:
R = cam['path'][0]['R']
T = cam['path'][0]['T']

In [27]:
T

[-1.1177868843078613, 0.8019675016403198, 1.45546555519104]

In [28]:
R_rotated, T_rotated = rotate_around_z(R, T, 90)

In [29]:
R_rotated

[0.4371144278739727,
 0.05875472460455873,
 0.8894843501083728,
 0.11955974060684663]

In [30]:
cam_rotated = cam.copy()
# cam_rotated['path'].append(cam['path'][0].copy())
# cam_rotated['path'][1]['R'] = R_rotated
# cam_rotated['path'][1]['T'] = T_rotated

In [31]:
# rotate by 90, 180, 270
for i in [90, 180, 270]:
    R_rotated, T_rotated = rotate_around_z(R, T, i)
    cam_rotated['path'].append(cam['path'][0].copy())
    cam_rotated['path'][-1]['R'] = R_rotated
    cam_rotated['path'][-1]['T'] = T_rotated

In [32]:
# save as cam_rotated.json
with open('../cam_rotated.json', 'w') as f:
    json.dump(cam_rotated, f, indent=4)